In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w))) # normal(均值, 标准差, 张量大小)
    y = torch.matmul(X, w) + b # matrix multiplication 涵盖了.mm & .mv, 并且可以批量操作
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000) # features 大小为(1000,2)

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) # 获取随机生成的数据

##### 我们可以调用框架中现有的API来读取数据。 我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据

In [3]:
# 这一部分取代了上节中手写的加载batch并且打乱的部分
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个pytorch数据迭代器
    dataset = data.TensorDataset(*data_arrays) # *代表参数解包, 把里面的参数拆开来给dataset
    return data.DataLoader(dataset, batch_size, shuffle = is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
# 总而言之 1.转化成tensordataset 2.用dataloader方法直接打乱 3.取出来变成了dataloader对象

In [4]:
next(iter(data_iter))

[tensor([[-0.5823,  0.7508],
         [-1.9926,  0.0915],
         [ 0.7648, -1.1810],
         [ 0.6267, -0.5525],
         [ 0.4599,  0.7956],
         [ 0.5357,  0.5585],
         [ 0.9634,  0.0394],
         [ 0.5642, -0.8818],
         [ 1.8082, -0.6265],
         [-1.5790, -0.0354]]),
 tensor([[ 0.4699],
         [-0.1033],
         [ 9.7315],
         [ 7.3195],
         [ 2.4256],
         [ 3.3859],
         [ 6.0029],
         [ 8.3216],
         [ 9.9483],
         [ 1.1674]])]

##### 对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 
##### 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 
##### 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

In [5]:
from torch import nn # nn means neural network

net = nn.Sequential(nn.Linear(2, 1))

##### 初始化模型参数.我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零
##### 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [6]:
# _操作起到一个节省内存的操作, 并且是原地修改变量, 不会影响梯度计算in_place
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [7]:
loss = nn.MSELoss() # mean square entropy loss 均方误差损失

In [8]:
# torch.optim 是优化器库
# SGD 随机梯度下降
# .parameter方法可以返回需要优化的参数，自动地（如权重，bias，embedding.etc）
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
list(net.parameters())

[Parameter containing:
 tensor([[0.0022, 0.0046]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True)]

In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        # net(X) 是整个网络最后的输出
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step() # trainer.step 是在更新参数，如何更新呢，这里的trainer定义的是SGD优化器,所以是sgd
    l = loss(net(features), labels)
    print(f'epoch{epoch + 1}, loss{l:f}')

epoch1, loss0.000288
epoch2, loss0.000104
epoch3, loss0.000105


In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0007, 0.0005])
b的估计误差： tensor([0.0004])
